In [1]:
import sys
from pathlib import Path

root_dir = Path.cwd().parent  # Go one folder up from /notebooks
print("Root directory:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))
    
import pandas as pd
df = pd.read_parquet("../data/enriched/financials_annual_ticker_sic.parquet")
df.head()

Root directory: /Users/agalyaayyadurai/Automated-Equity-Valuation


,adsh,cik,name,form,fy,fp,period,filed,sic,source_zip,...,WASODiluted,bs_diff,bs_balanced_flag,cf_delta_abs,cf_balanced_flag,coverage_score,WASOCombinedBasicDiluted,ticker,office,industry_title
0,0001047469-11-006302,1750,AAR CORP,10-K,2011,FY,20110531,2011-07-13,3720,2011q3,...,NaN,8.678820e+08,False,21964000.0,False,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS
1,0001047469-12-007300,1750,AAR CORP,10-K,2012,FY,20120531,2012-07-19,3720,2012q3,...,NaN,1.331004e+09,False,0.0,True,0.333333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS
2,0001047469-13-007797,1750,AAR CORP,10-K,2013,FY,20130531,2013-07-26,3720,2013q3,...,NaN,1.218300e+09,False,6400000.0,False,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS
3,0001047469-14-006243,1750,AAR CORP,10-K,2014,FY,20140531,2014-07-17,3720,2014q3,...,NaN,1.200000e+09,False,13000000.0,False,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS
4,0001047469-15-006136,1750,AAR CORP,10-K,2015,FY,20150531,2015-07-15,3720,2015q3,...,NaN,6.699000e+08,False,31900000.0,False,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS


## AR(1) model and OU Parameters

In [2]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss
import statsmodels.api as sm

# # 1) Keep only Tech & Healthcare offices
# tech_offices = ["Office of Technology"]
# health_offices = ["Office of Life Sciences"]

# df_sec = df[df["office"].isin(tech_offices + health_offices)].copy()

# # Treat office as sector label
# df_sec["sector"] = df_sec["office"].map({
#     "Office of Technology": "Technology",
#     "Office of Life Sciences": "Healthcare"
# })

#1) Keep only rows with a valid office and revenue
df_off = df.dropna(subset=["office", "Revenue"]).copy()
df_off = df_off[df_off["Revenue"] > 0].copy()

# 2) Ensure period is datetime & sort
df_off["period"] = pd.to_datetime(df_off["period"])
df_off = df_off.sort_values(["cik", "period"]).copy()

# 3) Compute log revenue & growth
df_off["log_rev"] = np.log(df_off["Revenue"])
df_off["log_rev_lag"] = df_off.groupby("cik")["log_rev"].shift(1)
df_off["g"] = df_off["log_rev"] - df_off["log_rev_lag"]

# Drop invalid growth
df_off = df_off.dropna(subset=["g"]).copy()

# 4) Keep only firms with enough history (>= 4 growth obs ≈ 5 years)
growth_counts = df_off.groupby("cik")["g"].size()
valid_ciks = growth_counts[growth_counts >= 4].index
df_off = df_off[df_off["cik"].isin(valid_ciks)].copy()

# 5) Build AR(1) lag variable
df_off["g_lag"] = df_off.groupby("cik")["g"].shift(1)
df_off = df_off.dropna(subset=["g_lag"]).copy()

# 6) Add year for office-level time series
df_off["year"] = df_off["period"].dt.year

df_off.head()


,adsh,cik,name,form,fy,fp,period,filed,sic,source_zip,...,coverage_score,WASOCombinedBasicDiluted,ticker,office,industry_title,log_rev,log_rev_lag,g,g_lag,year
2,0001047469-13-007797,1750,AAR CORP,10-K,2013,FY,2013-05-31,2013-07-26,3720,2013q3,...,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS,21.482809,21.448395,0.034414,0.150888,2013
3,0001047469-14-006243,1750,AAR CORP,10-K,2014,FY,2014-05-31,2014-07-17,3720,2014q3,...,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS,21.433762,21.482809,-0.049048,0.034414,2014
4,0001047469-15-006136,1750,AAR CORP,10-K,2015,FY,2015-05-31,2015-07-15,3720,2015q3,...,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS,21.189701,21.433762,-0.244061,-0.049048,2015
5,0001047469-16-014299,1750,AAR CORP,10-K,2016,FY,2016-05-31,2016-07-13,3720,2016q3,...,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS,21.231648,21.189701,0.041948,-0.244061,2016
6,0001047469-17-004528,1750,AAR CORP,10-K,2017,FY,2017-05-31,2017-07-12,3720,2017q3,...,0.583333,NaN,AIR,Office of Manufacturing,AIRCRAFT & PARTS,21.292889,21.231648,0.061240,0.041948,2017


In [3]:
from statsmodels.tsa.stattools import adfuller, kpss

def office_stationarity_tests(df):
    df = df.copy()
    df["year"] = df["period"].dt.year

    results = []

    for office, d in df.groupby("office"):
        # office-level annual average growth
        ts = (
            d.groupby("year")["g"]
             .mean()
             .sort_index()
        ).dropna()

        if len(ts) < 8:   # need enough years to say anything
            continue

        # ADF: H0 = unit root (non-stationary)
        adf_stat, adf_p, _, _, adf_crit, _ = adfuller(ts, autolag="AIC")

        # KPSS: H0 = stationary
        try:
            kpss_stat, kpss_p, _, kpss_crit = kpss(ts, regression="c", nlags="auto")
        except ValueError:
            kpss_stat, kpss_p, kpss_crit = np.nan, np.nan, {}

        results.append({
            "office": office,
            "n_years": len(ts),
            "adf_stat": adf_stat,
            "adf_pvalue": adf_p,
            "kpss_stat": kpss_stat,
            "kpss_pvalue": kpss_p,
        })

    return pd.DataFrame(results)

office_stationarity = office_stationarity_tests(df_off)
office_stationarity.sort_values("office", inplace=True)
print(office_stationarity)


                                               office  n_years  adf_stat  \
0                Industrial Applications and Services       15 -0.847384   
1                             Office of Crypto Assets       14 -0.930680   
2                   Office of Energy & Transportation       15 -4.932471   
3                                   Office of Finance       15 -2.472128   
4        Office of Finance or Office of Crypto Assets       12 -1.611950   
5        Office of Finance or Office of Crypto Assets       15 -2.875391   
6                             Office of Life Sciences       15 -5.148670   
7                             Office of Manufacturing       15 -3.693626   
8                Office of Real Estate & Construction       15 -2.618392   
9                                Office of Technology       15 -5.490453   
10                         Office of Trade & Services       15 -2.720793   
11  Office of Trade & Services or Office of Energy...       15 -1.534230   

    adf_pva

/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4039/272073087.py:25: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, _, kpss_crit = kpss(ts, regression="c", nlags="auto")
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4039/272073087.py:25: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, _, kpss_crit = kpss(ts, regression="c", nlags="auto")
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4039/272073087.py:25: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, _, kpss_crit = kpss(ts, regression="c", nlags="auto")
/var/folders/zz/l0v68lmx1_1fxy9

In [4]:
import statsmodels.api as sm

def estimate_ar1_by_office(df):
    results = []

    for office, d in df.groupby("office"):
        # avoid tiny groups
        if d["g_lag"].notnull().sum() < 30:
            continue

        X = sm.add_constant(d["g_lag"])
        y = d["g"]

        model = sm.OLS(y, X, missing="drop").fit()

        alpha = model.params["const"]
        phi = model.params["g_lag"]
        sigma = model.resid.std(ddof=1)

        kappa = 1 - phi
        mu = alpha / kappa if kappa != 0 else np.nan

        results.append({
            "office": office,
            "alpha": alpha,
            "phi (persistence)": phi,
            "kappa (speed of mean reversion)": kappa,
            "mu (long-run mean growth)": mu,
            "sigma (volatility)": sigma,
            "n_obs": len(d),
            "n_cik": d["cik"].nunique()
        })

    return pd.DataFrame(results)

office_params = estimate_ar1_by_office(df_off)
office_params.sort_values("office", inplace=True)
print(office_params)


                                               office     alpha  \
0                Industrial Applications and Services  0.085498   
1                             Office of Crypto Assets  0.122046   
2                   Office of Energy & Transportation  0.047066   
3                                   Office of Finance  0.053812   
4        Office of Finance or Office of Crypto Assets  0.063875   
5        Office of Finance or Office of Crypto Assets  0.092283   
6                             Office of Life Sciences  0.144572   
7                             Office of Manufacturing  0.046241   
8                Office of Real Estate & Construction  0.069384   
9                                Office of Technology  0.090271   
10                         Office of Trade & Services  0.068554   
11  Office of Trade & Services or Office of Energy...  0.116012   

    phi (persistence)  kappa (speed of mean reversion)  \
0           -0.116418                         1.116418   
1           

## Financials Data for Tech, Healthcare with their ticker

In [8]:
df_th = pd.read_parquet("../data/enriched/financials_annual_th_ticker.parquet")
df_th.head()

,adsh,cik,name,form,fy,fp,period,filed,sic,source_zip,...,cf_delta_abs,cf_balanced_flag,coverage_score,WASOCombinedBasicDiluted,ticker_old,office,industry_title,ticker,ticker_duplicates,ticker_instance
0,0001047469-10-001018,1800,ABBOTT LABORATORIES,10-K,2009,FY,20091231,2010-02-19,2834,2010q1,...,0.000000e+00,True,0.583333,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
1,0001047469-11-001056,1800,ABBOTT LABORATORIES,10-K,2010,FY,20101231,2011-02-18,2834,2011q1,...,1.327606e+10,False,0.750000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
2,0001047469-12-001216,1800,ABBOTT LABORATORIES,10-K,2011,FY,20111231,2012-02-21,2834,2012q1,...,0.000000e+00,True,0.500000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
3,0001047469-13-001180,1800,ABBOTT LABORATORIES,10-K,2012,FY,20121231,2013-02-15,2834,2013q1,...,0.000000e+00,True,0.500000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
4,0001047469-14-001176,1800,ABBOTT LABORATORIES,10-K,2013,FY,20131231,2014-02-21,2834,2014q1,...,7.301000e+09,False,0.833333,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None


In [19]:
#Remove all rows that have ticker as "DELISTED"
df_th = df_th[~(df_th["ticker"] == "DELISTED")]
df_th

,adsh,cik,name,form,fy,fp,period,filed,sic,source_zip,...,cf_delta_abs,cf_balanced_flag,coverage_score,WASOCombinedBasicDiluted,ticker_old,office,industry_title,ticker,ticker_duplicates,ticker_instance
0,0001047469-10-001018,1800,ABBOTT LABORATORIES,10-K,2009,FY,20091231,2010-02-19,2834,2010q1,...,0.000000e+00,True,0.583333,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
1,0001047469-11-001056,1800,ABBOTT LABORATORIES,10-K,2010,FY,20101231,2011-02-18,2834,2011q1,...,1.327606e+10,False,0.750000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
2,0001047469-12-001216,1800,ABBOTT LABORATORIES,10-K,2011,FY,20111231,2012-02-21,2834,2012q1,...,0.000000e+00,True,0.500000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
3,0001047469-13-001180,1800,ABBOTT LABORATORIES,10-K,2012,FY,20121231,2013-02-15,2834,2013q1,...,0.000000e+00,True,0.500000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
4,0001047469-14-001176,1800,ABBOTT LABORATORIES,10-K,2013,FY,20131231,2014-02-21,2834,2014q1,...,7.301000e+09,False,0.833333,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,ABT,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18630,0000950170-25-078416,2018064,"TECHTARGET, INC.",10-K,2024,FY,20241231,2025-05-28,4822,2025q2,...,2.654160e+08,False,1.000000,NaN,NaN,Office of Technology,TELEGRAPH & OTHER MESSAGE COMMUNICATIONS,TTGT,None,None
18631,0001410578-25-000432,2020795,"WINDSTREAM PARENT, INC.",10-K,2024,FY,20241231,2025-03-24,4813,2025q1,...,0.000000e+00,True,0.083333,NaN,NaN,Office of Technology,TELEPHONE COMMUNICATIONS (NO RADIOTELEPHONE),UNIT,None,None
18633,0000950170-25-037621,2022626,"UPSTREAM BIO, INC.",10-K,2024,FY,20241231,2025-03-12,2834,2025q1,...,3.002530e+08,False,1.000000,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,UPB,None,None
18634,0002023658-25-000012,2023658,BICARA THERAPEUTICS INC.,10-K,2024,FY,20241231,2025-03-27,2834,2025q1,...,2.592710e+08,False,0.916667,NaN,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,BCAX,None,None


In [22]:
import pandas as pd
#Clean filed and period to proper datetimes and flag invalids
# Make a copy if you like
df = df_th.copy()

# Convert to datetime, invalid -> NaT
df["filed_dt"] = pd.to_datetime(df["filed"], errors="coerce")
df["period_dt"] = pd.to_datetime(df["period"], errors="coerce")

# Keep only dates between 2000 and 2025 (others -> NaT)
for col in ["filed_dt", "period_dt"]:
    valid_year = df[col].dt.year.between(2000, 2025)
    df.loc[~valid_year, col] = pd.NaT

# Optional: inspect invalid ones
invalid_rows = df[df["filed_dt"].isna() | df["period_dt"].isna()]
print("Rows with invalid filed/period:", len(invalid_rows))
print(invalid_rows[["cik", "filed", "period"]].head())

Rows with invalid filed/period: 0
Empty DataFrame
Columns: [cik, filed, period]
Index: []


In [24]:
df.to_csv("../data/check_date.csv")

In [23]:
# Difference in days: filed - period
df["filed_minus_period"] = (df["filed_dt"] - df["period_dt"]).dt.days

# Ignore rows where either is NaT
valid_diff = df["filed_minus_period"].dropna()

max_diff = valid_diff.max()
print("Max (filed - period) difference in days:", max_diff)

Max (filed - period) difference in days: 4018


In [29]:
df["filed_minus_period"] = df["filed_minus_period"].astype(int)
df[df["filed_minus_period"] > 300]

,adsh,cik,name,form,fy,fp,period,filed,sic,source_zip,...,ticker_old,office,industry_title,ticker,ticker_duplicates,ticker_instance,filed_dt,filed_valid,period_dt,filed_minus_period
17,0001264931-15-000058,1961,WORLDS INC,10-K/A,2012,FY,20121231,2015-02-06,7372,2015q1,...,NaN,Office of Technology,SERVICES-PREPACKAGED SOFTWARE,WDDD,None,"[WDDD, WDDD10K24_HTM.XML, WDDDK21_HTM.XML, WDD...",2015-02-06,True,2012-12-31,767
18,0001264931-15-000067,1961,WORLDS INC,10-K/A,2013,FY,20131231,2015-02-09,7372,2015q1,...,NaN,Office of Technology,SERVICES-PREPACKAGED SOFTWARE,WDDD,None,"[WDDD, WDDD10K24_HTM.XML, WDDDK21_HTM.XML, WDD...",2015-02-09,True,2013-12-31,405
481,0000047217-23-000075,47217,HP INC,10-K/A,2022,FY,20221031,2023-09-11,3570,2023q3,...,NaN,Office of Technology,COMPUTER & OFFICE EQUIPMENT,HPQ,None,None,2023-09-11,True,2022-10-31,315
792,0000078749-16-000068,78749,AGILYSYS INC,10-K/A,2014,FY,20150331,2016-02-09,7373,2016q1,...,NaN,Office of Technology,SERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN,AGYS,None,None,2016-02-09,True,2015-03-31,315
841,0001017386-16-000646,90721,PERVASIP CORP,10-K,2015,FY,20151130,2016-12-15,4813,2016q4,...,NaN,Office of Technology,TELEPHONE COMMUNICATIONS (NO RADIOTELEPHONE),PVSP,None,[PVSP],2016-12-15,True,2015-11-30,381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17416,0001786352-23-000037,1786352,"BILL HOLDINGS, INC.",10-K/A,2022,FY,20220630,2023-05-26,7372,2023q2,...,NaN,Office of Technology,SERVICES-PREPACKAGED SOFTWARE,BILL,None,None,2023-05-26,True,2022-06-30,330
17692,0001654954-22-005746,1815632,"TEGO CYBER, INC.",10-K/A,2021,FY,20210630,2022-05-02,7370,2022q2,...,NaN,Office of Technology,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",TGCB,None,None,2022-05-02,True,2021-06-30,306
17709,0000950170-25-030856,1816017,"SPIRE GLOBAL, INC.",10-K/A,2023,FY,20231231,2025-03-03,4899,2025q1,...,NaN,Office of Technology,"COMMUNICATIONS SERVICES, NEC",SPIR,None,None,2025-03-03,True,2023-12-31,428
17830,0001213900-21-065246,1821586,HELIX ACQUISITION CORP,10-K/A,2020,FY,20201231,2021-12-14,2834,2021q4,...,NaN,Office of Life Sciences,PHARMACEUTICAL PREPARATIONS,MLTX,None,None,2021-12-14,True,2020-12-31,348


In [13]:
import pandas as pd
import yfinance as yf

def get_first_trading_ohlcv(row):
    ticker = row["ticker"]
    filed  = row["filed"]   # change if your column has a different name

    if pd.isna(ticker) or pd.isna(filed):
        return pd.Series(
            [pd.NaT, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA],
            index=["trade_date", "open", "high", "low", "close", "volume"],
        )

    start = pd.to_datetime(filed)
    end   = start + pd.Timedelta(days=7)  # search up to 7 days ahead for safety

    data = yf.Ticker(ticker).history(start=start, end=end)

    if data.empty:
        return pd.Series(
            [pd.NaT, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA],
            index=["trade_date", "open", "high", "low", "close", "volume"],
        )

    first_day = data.index[0]
    first_row = data.iloc[0]

    return pd.Series(
        [
            first_day,
            first_row["Open"],
            first_row["High"],
            first_row["Low"],
            first_row["Close"],
            first_row["Volume"],
        ],
        index=["trade_date", "open", "high", "low", "close", "volume"],
    )

# Apply once, get all 6 new columns in df_th
df_th[["trade_date", "open", "high", "low", "close", "volume"]] = df_th.apply(
    get_first_trading_ohlcv, axis=1
)


HTTP Error 404: 
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly delisted; no timezone found
$DELISTED: possibly del

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [31]:
df_th["form"].value_counts()

form
10-K      12087
10-K/A      549
Name: count, dtype: int64

## Data Cleaning and Data Integrity

In [6]:
import pandas as pd

# Ensure datetime
df_th["period"] = pd.to_datetime(df_th["period"], errors="coerce")
df_th["filed"]  = pd.to_datetime(df_th["filed"],  errors="coerce")

# Check invalid cases
invalid_mask = df_th["filed"] < df_th["period"]

invalid_rows = df_th[invalid_mask].copy()

print("Total records:", df_th.shape[0])
print("Invalid filing < period:", invalid_rows.shape[0])
print("Percentage:", round(invalid_rows.shape[0] / df_th.shape[0], 4))

# Preview a few
invalid_rows[["cik", "ticker", "name", "period", "filed", "office"]].head(20)


Total records: 18637
Invalid filing < period: 0
Percentage: 0.0


,cik,ticker,name,period,filed,office


## Attach OHLCV data to the tech healthcare fundamentals

In [3]:
from src.data_enrich.ohlcv_to_panel import attach_ohlcv_to_fundamentals

df_ohlcv = attach_ohlcv_to_fundamentals("../data/enriched/financials_annual_th_ticker.parquet", "../data/enriched/financials_annual_th_ohlcv.parquet")

Anchor date range: 2009-08-22 00:00:00 → 2025-07-01 00:00:00
Number of unique tickers to download: 1701


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end, progress=False, group_by="column")


AACS: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'AACS'), ('High', 'AACS'), ('Low', 'AACS'), ('Open', 'AACS'), ('Volume', 'AACS')]
AAGH: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'AAGH'), ('High', 'AAGH'), ('Low', 'AAGH'), ('Open', 'AAGH'), ('Volume', 'AAGH')]


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end, progress=False, group_by="column")
/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end, progress=False, group_by="column")


AAON: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'AAON'), ('High', 'AAON'), ('Low', 'AAON'), ('Open', 'AAON'), ('Volume', 'AAON')]


...

ZVRA: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'ZVRA'), ('High', 'ZVRA'), ('Low', 'ZVRA'), ('Open', 'ZVRA'), ('Volume', 'ZVRA')]
ZVSA: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'ZVSA'), ('High', 'ZVSA'), ('Low', 'ZVSA'), ('Open', 'ZVSA'), ('Volume', 'ZVSA')]


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end, progress=False, group_by="column")
/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end, progress=False, group_by="column")


ZWS: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'ZWS'), ('High', 'ZWS'), ('Low', 'ZWS'), ('Open', 'ZWS'), ('Volume', 'ZWS')]
ZYME: No Adj Close/Close column found in downloaded data. Columns: [('Date', ''), ('Close', 'ZYME'), ('High', 'ZYME'), ('Low', 'ZYME'), ('Open', 'ZYME'), ('Volume', 'ZYME')]


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t, start=start, end=end, progress=False, group_by="column")


RuntimeError: No price data downloaded for any ticker. Check tickers and yfinance connection.

In [2]:
from src.data_enrich.ohlcv_to_panel import debug_yfinance_and_tickers, attach_ohlcv_to_fundamentals

debug_df = debug_yfinance_and_tickers(
    "../data/enriched/financials_annual_th_ticker.parquet",
    ticker_col="ticker",
    n_sample=5,
)

Total rows (non-null, non-DELISTED tickers): 12636
Number of unique tickers: 1701

Sample tickers from file:
['ABT', 'WDDD', 'CECO', 'AP', 'ANLY', 'EPAC', 'ARTW', 'ALOT', 'ADP', 'DOMH', 'BMYMP', 'BC', 'CACI', 'CASC', 'CAT', 'LUMN', 'CVR', 'FYBR', 'TXMD', 'CMI']

Testing yfinance with AAPL (connectivity check)...


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  test_aapl = yf.download("AAPL", period="5d", progress=False, group_by="column")


AAPL data shape: (5, 5)
✅ AAPL returned data. yfinance connectivity seems OK.

Testing yfinance with 5 tickers from your file: ['ABT', 'WDDD', 'CECO', 'AP', 'ANLY']
Ticker ABT: shape (5, 5)


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="5d", progress=False, group_by="column")
/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="5d", progress=False, group_by="column")


Ticker WDDD: shape (4, 5)
Ticker CECO: shape (5, 5)


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="5d", progress=False, group_by="column")
/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="5d", progress=False, group_by="column")


Ticker AP: shape (5, 5)


/Users/agalyaayyadurai/Automated-Equity-Valuation/src/data_enrich/ohlcv_to_panel.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="5d", progress=False, group_by="column")

1 Failed download:
['ANLY']: YFPricesMissingError('possibly delisted; no price data found  (period=5d)')


Ticker ANLY: shape (0, 6)


In [4]:
debug_df[debug_df["ticker"=="AAPL"]]

KeyError: False

In [5]:
df_th = pd.read_parquet("../data/enriched/financials_annual_th_ticker.parquet")
df_th.head()

unique_tickers = df_th["ticker"].dropna().unique().tolist()
print(unique_tickers[:50])

import yfinance as yf

for t in unique_tickers[:20]:
    df_t = yf.download(t, period="1mo")
    print(t, df_t.shape)


['ABT', 'WDDD', 'DELISTED', 'CECO', 'AP', 'ANLY', 'EPAC', 'ARTW', 'ALOT', 'ADP', 'DOMH', 'BMYMP', 'BC', 'CACI', 'CASC', 'CAT', 'LUMN', 'CVR', 'FYBR', 'TXMD', 'CMI', 'CW', 'USAU', 'DBD', 'DCI', 'DOV', 'FLS', 'ETN', 'TENX', 'TGNA', 'GRC', 'GGG', 'GTN-A', 'HPQ', 'ITW', 'TRAK', 'IBM', 'KMT', 'LLY', 'LECO', 'LUFK', 'MTW', 'SLNHP', 'GENC', 'MRK', 'MPR', 'MOG-A', 'CTDD', 'NTY', 'VYX']


/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")
[*********************100%***********************]  1 of 1 completed
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")
[*********************100%***********************]  1 of 1 completed
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")


ABT (22, 5)
WDDD (21, 5)


HTTP Error 404: 
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['DELISTED']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")
[*********************100%***********************]  1 of 1 completed
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")


DELISTED (0, 6)
CECO (22, 5)


...

CAT (22, 5)
LUMN (22, 5)


[*********************100%***********************]  1 of 1 completed
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")
[*********************100%***********************]  1 of 1 completed
/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_4540/2753151415.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_t = yf.download(t, period="1mo")


CVR (22, 5)
FYBR (22, 5)


[*********************100%***********************]  1 of 1 completed

TXMD (22, 5)


In [7]:
df_th["ticker"].unique()

array(['ABT', 'WDDD', 'DELISTED', ..., 'UPB', 'BCAX', 'MTSR'],
      dtype=object)